# Performance of time-series perdictions of M4 competition

In M4 competition, there are 100000 time-series. The data is grouped into the different periods ("Hourly", "Daily", "Weekly", "Monthly", "Quarterly", "Yearly").
It is required to forcast from 6 steps to 48 steps. The dataset can be found in the following link:
https://github.com/Mcompetitions/M4-methods/tree/master/Dataset

We perform the time-series forecasting using the transformer model with the standard approach.
<li> The features are generated from a univariate time-series (e.g. taking lags, taking square and cubic, etc.)
<li> We keep 20 features with the highest correlation with the original time-series.
<li> Due to the limited resource, we fix the model parameters without hyperparameter tuning.

From our result, the best performance is to predict the hourly time-series. The simulation time is long due to using the deep leanring model. The performance may be increased by using a more complex model and with the appropriate hyperparameter tuning. We also provide the other types of deep learning models(CNN1D, CNN2D, LSTM, Bidirectional-LSTM, CNN1D-LSTM and CNN1D-BiLSTM).
We did some preliminary studies on the dataset using those models. However, it would take long time to complete all the simulations. From the preliminary study, we found that the transformer model outperforms the other deep learning models on time-series forecasting for M4 dataset.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dict = {}
period_list = ["Hourly", "Daily", "Weekly", "Monthly", "Quarterly", "Yearly"]

for period in period_list:
    train = pd.read_csv("Dataset/Train/{}-train.csv".format(period), index_col=0).T
    test = pd.read_csv("Dataset/Test/{}-test.csv".format(period), index_col=0).T
    column = train.columns
    train_length = []
    test_length = []
    for i in column:
        train_length.append(len(train[i].dropna()))
        test_length.append(len(test[i].dropna()))
    data_dict[period] = [len(column), max(train_length), round(np.mean(train_length),2), min(train_length), max(test_length)]
    
data_df = pd.DataFrame().from_dict(data_dict).astype(int)
data_df.index = ["Number of time-series", "Max. number of train data", "Mean number of train data",
                "Min. number of test data", "Number of test data"]

In [3]:
def data_combine(period, folder):
    data = pd.DataFrame()
    filelist = os.listdir(folder)
    for i in filelist:
        if period in i:
            path = os.path.join(folder,i)
            data = pd.read_csv(path, index_col=0)
    return data

In [7]:
result_dict = {}
result_df = pd.DataFrame()
overall = pd.DataFrame()
for period in period_list:
    result_dict[period] = data_combine(period, "result")
    overall = pd.concat([overall, result_dict[period]], axis=0)
    result_df[period] = result_dict[period].mean().round(2)
result_df["Overall"] = overall.mean().round(2)

# Data 

We outline the number of time-series in each period and the numbers of data in train and test data, respectively.

In [4]:
data_df

,Hourly,Daily,Weekly,Monthly,Quarterly,Yearly
Number of time-series,414,4227,359,48000,24000,23000
Max. number of train data,960,9919,2597,2794,866,835
Mean number of train data,853,2357,1022,216,92,31
Min. number of test data,700,93,80,42,16,13
Number of test data,48,14,13,18,8,6


# Prediction result

We adopt the metrics, SMAPE, MASE, OWA, MAE and RMSE to measure the performance of the predictions of each time-series. We take the average of each metrics as follows:

In [8]:
result_df

,Hourly,Daily,Weekly,Monthly,Quarterly,Yearly,Overall
SMAPE,12.40,13.64,27.76,30.09,40.04,53.81,37.16
MASE,2.10,15.71,20.61,3.49,6.30,13.01,6.93
OWA,0.68,11.57,15.47,8.74,9.44,6.29,8.45
MAE,548.81,822.69,1378.08,1404.28,2193.40,2841.42,1895.99
RMSE,611.40,840.56,1428.40,1510.82,2270.64,2928.27,1986.84
